In [1]:
# Import libraries
import pandas as pd
import re
import requests as req
from bs4 import BeautifulSoup as bs
import json
import warnings
warnings.simplefilter('ignore')
from sqlalchemy import create_engine

In [2]:
# Cleaning functions (py file) placed in src folder.
import sys
sys.path.append("../")
from src.cleaning import *

# Import data

In [3]:
# Import data
df=pd.read_csv("../data/Artworks.csv")

df.head(3)

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,...,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0s...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,Paint and colored pencil on print,...,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN
2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,"Graphite, pen, color pencil, ink, and gouache ...",...,http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...,NaN,NaN,NaN,34.3000,NaN,NaN,31.8000,NaN,NaN


# Data examination

In [4]:
artist = list(df.Artist.unique())

In [5]:
len(artist)

13863

In [6]:
df.shape

(132403, 29)

In [7]:
df.isnull().sum()

Title                     45
Artist                  1461
ConstituentID           1461
ArtistBio               4734
Nationality             1461
BeginDate               1461
EndDate                 1461
Gender                  1461
Date                    2166
Medium                 11225
Dimensions             11243
CreditLine              2924
AccessionNumber            0
Classification             0
Department                 0
DateAcquired            5994
Cataloged                  0
ObjectID                   0
URL                    57280
ThumbnailURL           68263
Circumference (cm)    132393
Depth (cm)            120479
Diameter (cm)         130985
Height (cm)            18765
Length (cm)           131665
Weight (kg)           132110
Width (cm)             19664
Seat Height (cm)      132403
Duration (sec.)       129258
dtype: int64

# CLEANING

Drop columns non useful

In [8]:
# DROP columns that are not useful for my analysis.
df1 = df.drop(["Title", "ConstituentID", "ArtistBio","Medium","ThumbnailURL", "BeginDate", "Gender", "EndDate", "Circumference (cm)", "Depth (cm)", "Diameter (cm)", "Height (cm)", "Length (cm)", "Weight (kg)", "Width (cm)", "Seat Height (cm)", "Duration (sec.)", "Dimensions", "CreditLine", "AccessionNumber", "Department", "DateAcquired", "Cataloged", "ObjectID", "URL"],axis=1)

In [9]:
df1.sample()

,Artist,Nationality,Date,Classification
101355,George Maciunas,(American),NaN,(not assigned)


Cleaning column date

In [10]:
# Cleaning "date" column to obtain an integer year.
date = list(df1.Date.unique())

In [11]:
date[:5]

['1896', '1987', '1903', '1980', '1976-77']

In [12]:
df1['year'] = df1['Date'].str.extract(r"(\d+)")

In [13]:
df1.head(3)

,Artist,Nationality,Date,Classification,year
0,Otto Wagner,(Austrian),1896,Architecture,1896
1,Christian de Portzamparc,(French),1987,Architecture,1987
2,Emil Hoppe,(Austrian),1903,Architecture,1903


In [14]:
# Dropping column "Date"; I am going to use "year" column.
df1 = df1.drop(["Date"],axis=1)

In [15]:
df1.sample()

,Artist,Nationality,Classification,year
105423,Carl Otto Czeschka,(Austrian),Illustrated Book,1920


In [16]:
# Drop rows that have NaN in year column.
df1 = df1[df1['year'].notna()]

In [17]:
"""def integrar(x):
        return int(x)"""

'def integrar(x):\n        return int(x)'

In [18]:
integrar

<function src.cleaning.integrar(x)>

In [19]:
df1["year"] = df1.year.apply(integrar)

In [20]:
# I focus on the exhibitions from the last 50 years.
df2 = df1[df1["year"]>=1972]

In [21]:
df2.shape

(42319, 4)

In [22]:
# Elimino los NaN en la columna artist.
df2 = df2[df2['Artist'].notna()]

In [23]:
artist = list(df2.Artist)

In [24]:
artist[:5]

['Christian de Portzamparc',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi']

Cleaning column Artist.

In [25]:
# I first make a split on ",", and after that I make an "explode".

"""def split(x):
    return x.split(",")"""
#df2['artist'] = df2['Artist'].str.split(",")

'def split(x):\n    return x.split(",")'

In [26]:
split

<function src.cleaning.split(x)>

In [27]:
df2["artist"] = df2.Artist.apply(split)

In [28]:
art = list(df2.artist)

In [29]:
art[:5]

[['Christian de Portzamparc'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi']]

Cleaning column Country

In [30]:
#LIMPIEZA columna country.
"""def split2(x):
    return x.split(" ")"""
# df2['Country'] = df2['Nationality'].str.split(" ")

'def split2(x):\n    return x.split(" ")'

In [31]:
split2

<function src.cleaning.split2(x)>

In [32]:
df2["Countries"] = df2.Nationality.apply(split2)

In [33]:
df3 = df2.explode('artist').explode('Countries')

In [34]:
"""def strip(x):
    return x.strip()"""

'def strip(x):\n    return x.strip()'

In [35]:
strip

<function src.cleaning.strip(x)>

In [36]:
df3["artist"] = df3.artist.apply(strip)

In [37]:
art = list((df3.artist).unique())

In [38]:
#At the end I obtain this list.
art[:5]

['Christian de Portzamparc',
 'Bernard Tschumi',
 'Peter Eisenman',
 'Robert Cole',
 'Rem Koolhaas']

In [39]:
df3 = df3.drop(["Artist"],axis=1)

In [40]:
df3.head(3)

,Nationality,Classification,year,artist,Countries
1,(French),Architecture,1987,Christian de Portzamparc,(French)
3,(),Architecture,1980,Bernard Tschumi,()
5,(),Architecture,1976,Bernard Tschumi,()


In [41]:
list_countries = list(df3.Countries)
list_countries[:5]

['(French)', '()', '()', '()', '()']

In [ ]:
#Change Nationality for country.
#df3['country'] = df3['Country'].replace("(French)", "France").replace("(American)", "United States").replace("(Italian)", "Italy").replace("(Dutch)", "Netherlands").replace("(British)", "United Kingdom").replace("(Japanese)", "Japan").replace("(Argentine)", "Argentina").replace("(Swiss)", "Switzerland").replace("(Luxembourgish)", "Luxembourg").replace("(Spanish)", "Spain").replace("(Polish)", "Poland").replace("(Austrian)", "Austria").replace("(Iranian)", "Iran").replace("(German)", "Germany").replace("(Moroccan)", "Morocco").replace("(Danish)", "Denmark").replace("(Canadian)", "Canada").replace("(Brazilian)", "Brazil").replace("(Venezuelan)", "Venezuela").replace("(Belgian)", "Belgium").replace("(Norwegian)", "Norway").replace("(Finnish)", "Finland").replace("(Swedish)", "Sweden").replace("(Colombian)", "Colombia").replace("(Australian)", "Autralia").replace("(Yugoslav)", "Yugoslavia").replace("(Nationality", "None").replace("unknown)", "None").replace("(Hungarian)", "Hungary").replace("(Mexican)", "Mexico").replace("(Greek)", "Greece").replace("(Israeli)", "Israel").replace("(Croatian)", "Croatia").replace("(Cuban)", "Cuba").replace("(Thai)", "Thailand"). replace("(Czech)", "Czech Republic").replace("(Chilean)", "Chile").replace("(Various)", "None").replace("(Romanian)", "Romania").replace("(South", "South Africa").replace("African)", "South Africa").replace("(Russian)", "Russian Federation").replace("(Congolese)", "Central African Republic").replace("(Ukrainian)", "Ukraine").replace("(Peruvian)", "Peru").replace("(Indian)", "India").replace("(Cambodian)", "Cambodia").replace("(Haitian)", "Haiti").replace("(Scottish)", "United Kingdom").replace("(Korean)", "South Korea").replace("(Slovak)", "Slovak Republic").replace("(Estonian)", "Estonia").replace("(Pakistani)", "Pakistan").replace("(Icelandic)", "Iceland").replace("(Portuguese)", "Portugal").replace("(Chinese)", "China").replace("(Paraguayan)", "Paraguay").replace("(Uruguayan)", "Uruguay").replace("(Tunisian)", "Tunisia").replace("(Guyanese)", "Guatemala").replace("(Senegalese)", "Senegal").replace("(Bahamian)", "Cuba").replace("(Turkish)", "Turkey").replace("(Malian)", "Mali").replace("(Bulgarian)", "Bulgaria").replace("(New", "New Zealand").replace("Zealander)", "New Zealand").replace("(Irish)", "Ireland").replace("(Lebanese)", "Lebanon").replace("(English)", "United Kingdom"). replace("(Welsh)", "United Kingdom").replace("(Cypriot)", "Cyprus").replace("(Kenyan)", "Kenya").replace("(Syrian)", "Syrian Arab Republic").replace("(Saudi", "Saudi Arabia").replace("Arabian)", "Saudi Arabia").replace("(Slovenian)", "Slovenia").replace("(Nigerian)", "Central African Republic").replace("(Native", "United States").replace("American)", "United States").replace("(Bosnian)","Bosnia and Herzegovina").replace("Unknown)", "None").replace("(Kazakhstani)", "Kazakhstan").replace("(Kyrgyzstani)", "Kyrgyz Republic").replace("(Palestinian)", "Israel").replace("(Catalan)", "Spain").replace("(Vietnamese)", "Vietnam").replace("(Indonesian)", "Indonesia").replace("(Filipino)", "Philippines").replace("(Guatemalan)", "Guatemala").replace("(Egyptian)", "Egypt").replace("(Ghanaian)", "Ghana").replace("(Serbian)", "Bosnia and Herzegovina").replace("(Mauritanian)", "Mauritania").replace("(Ugandan)", "Uganda").replace("(Taiwanese)", "Taiwan").replace("(Albanian)", "Albania").replace("(Malaysian)", "Malaysia").replace("(Iraqi)", "Iraq").replace("()", "None").replace("(Tajik)", "Tajikistan").replace("(Singaporean)", "Singapore").replace("(Puerto", "Dominican Republic").replace("Rican)","Dominican Republic").replace("(Zimbabwean)", "Zimbabwe").replace("(Cameroonian)", "Cameroon").replace("(Rwandan)", "Rwanda").replace("(Mozambican)", "Mozambique").replace("(Macedonian)", "Greece").replace("(Georgian)", "None").replace("(Sudanese)", "Sudan")

In [ ]:
"""def replace(x):
    return x.replace("(French)", "France").replace("(American)", "United States").replace("(Italian)", "Italy")
    .replace("(Dutch)", "Netherlands").replace("(British)", "United Kingdom").replace("(Japanese)", "Japan")
    .replace("(Argentine)", "Argentina").replace("(Swiss)", "Switzerland").replace("(Luxembourgish)", "Luxembourg")
    .replace("(Spanish)", "Spain").replace("(Polish)", "Poland").replace("(Austrian)", "Austria").replace("(Iranian)", "Iran")
    .replace("(German)", "Germany").replace("(Moroccan)", "Morocco").replace("(Danish)", "Denmark").
    replace("(Canadian)", "Canada").replace("(Brazilian)", "Brazil").replace("(Venezuelan)", "Venezuela")
    .replace("(Belgian)", "Belgium").replace("(Norwegian)", "Norway").replace("(Finnish)", "Finland")
    .replace("(Swedish)", "Sweden").replace("(Colombian)", "Colombia").replace("(Australian)", "Autralia")
    .replace("(Yugoslav)", "Yugoslavia").replace("(Nationality", "None").replace("unknown)", "None")
    .replace("(Hungarian)", "Hungary").replace("(Mexican)", "Mexico").replace("(Greek)", "Greece")
    .replace("(Israeli)", "Israel").replace("(Croatian)", "Croatia").replace("(Cuban)", "Cuba")
    .replace("(Thai)", "Thailand"). replace("(Czech)", "Czech Republic").replace("(Chilean)", "Chile")
    .replace("(Various)", "None").replace("(Romanian)", "Romania").replace("(South", "South Africa")
    .replace("African)", "South Africa").replace("(Russian)", "Russian Federation")
    .replace("(Congolese)", "Central African Republic").replace("(Ukrainian)", "Ukraine").replace("(Peruvian)", "Peru")
    .replace("(Indian)", "India").replace("(Cambodian)", "Cambodia").replace("(Haitian)", "Haiti")
    .replace("(Scottish)", "United Kingdom").replace("(Korean)", "South Korea").replace("(Slovak)", "Slovak Republic")
    .replace("(Estonian)", "Estonia").replace("(Pakistani)", "Pakistan").replace("(Icelandic)", "Iceland")
    .replace("(Portuguese)", "Portugal").replace("(Chinese)", "China").replace("(Paraguayan)", "Paraguay")
    .replace("(Uruguayan)", "Uruguay").replace("(Tunisian)", "Tunisia").replace("(Guyanese)", "Guatemala")
    .replace("(Senegalese)", "Senegal").replace("(Bahamian)", "Cuba").replace("(Turkish)", "Turkey")
    .replace("(Malian)", "Mali").replace("(Bulgarian)", "Bulgaria").replace("(New", "New Zealand")
    .replace("Zealander)", "New Zealand").replace("(Irish)", "Ireland").replace("(Lebanese)", "Lebanon")
    .replace("(English)", "United Kingdom"). replace("(Welsh)", "United Kingdom").replace("(Cypriot)", "Cyprus")
    .replace("(Kenyan)", "Kenya").replace("(Syrian)", "Syrian Arab Republic").replace("(Saudi", "Saudi Arabia")
    .replace("Arabian)", "Saudi Arabia").replace("(Slovenian)", "Slovenia").replace("(Nigerian)", "Central African Republic")
    .replace("(Native", "United States").replace("American)", "United States").replace("(Bosnian)","Bosnia and Herzegovina")
    .replace("Unknown)", "None").replace("(Kazakhstani)", "Kazakhstan").replace("(Kyrgyzstani)", "Kyrgyz Republic")
    .replace("(Palestinian)", "Israel").replace("(Catalan)", "Spain").replace("(Vietnamese)", "Vietnam")
    .replace("(Indonesian)", "Indonesia").replace("(Filipino)", "Philippines").replace("(Guatemalan)", "Guatemala")
    .replace("(Egyptian)", "Egypt").replace("(Ghanaian)", "Ghana").replace("(Serbian)", "Bosnia and Herzegovina")
    .replace("(Mauritanian)", "Mauritania").replace("(Ugandan)", "Uganda").replace("(Taiwanese)", "Taiwan")
    .replace("(Albanian)", "Albania").replace("(Malaysian)", "Malaysia").replace("(Iraqi)", "Iraq").replace("()", "None")
    .replace("(Tajik)", "Tajikistan").replace("(Singaporean)", "Singapore").replace("(Puerto", "Dominican Republic")
    .replace("Rican)","Dominican Republic").replace("(Zimbabwean)", "Zimbabwe").replace("(Cameroonian)", "Cameroon")
    .replace("(Rwandan)", "Rwanda").replace("(Mozambican)", "Mozambique").replace("(Macedonian)", "Greece")
    .replace("(Georgian)", "None").replace("(Sudanese)", "Sudan")"""

In [42]:
replace

<function src.cleaning.replace(x)>

In [43]:
df3["country"] = df3.Countries.apply(replace)

In [44]:
list_country = list((df3['country']).unique())

In [45]:
list_country[:5]

['France', 'None', 'United States', 'Netherlands', 'Italy']

In [46]:
df3 = df3.drop(["Countries"],axis=1)

In [47]:
df3 = df3.drop(["Nationality"],axis=1)

In [48]:
df3

,Classification,year,artist,country
1,Architecture,1987,Christian de Portzamparc,France
3,Architecture,1980,Bernard Tschumi,None
5,Architecture,1976,Bernard Tschumi,None
6,Architecture,1976,Bernard Tschumi,None
7,Architecture,1976,Bernard Tschumi,None
...,...,...,...,...
132308,Print,2016,Christian Vinck Henriquez,Venezuela
132309,Multiple,2016,Christian Vinck Henriquez,Venezuela
132331,Illustrated Book,2015,Laura Owens,United States
132332,Illustrated Book,2015,Laura Owens,United States


In [49]:
df3.drop(df3.loc[df3['country']=='None'].index, inplace=True)

In [50]:
df4= df3.drop_duplicates()

In [51]:
df4

,Classification,year,artist,country
1,Architecture,1987,Christian de Portzamparc,France
66,Architecture,1987,Rem Koolhaas,Netherlands
66,Architecture,1987,Madelon Vriesendorp,Netherlands
67,Architecture,1979,Roger C. Ferri,United States
72,Architecture,1978,Michael Graves,United States
...,...,...,...,...
132284,Multiple,2016,Nicolás Paris,Colombia
132288,Illustrated Book,2016,Nicolás Paris,Colombia
132289,Print,2016,Rosângela Rennó,Brazil
132290,Print,2016,Christian Vinck Henriquez,Venezuela


In [52]:
df4.reset_index()

,index,Classification,year,artist,country
0,1,Architecture,1987,Christian de Portzamparc,France
1,66,Architecture,1987,Rem Koolhaas,Netherlands
2,66,Architecture,1987,Madelon Vriesendorp,Netherlands
3,67,Architecture,1979,Roger C. Ferri,United States
4,72,Architecture,1978,Michael Graves,United States
...,...,...,...,...,...
17237,132284,Multiple,2016,Nicolás Paris,Colombia
17238,132288,Illustrated Book,2016,Nicolás Paris,Colombia
17239,132289,Print,2016,Rosângela Rennó,Brazil
17240,132290,Print,2016,Christian Vinck Henriquez,Venezuela


In [53]:
df4.to_csv("data/moma.csv", index = False)